In [2]:
%matplotlib inline
# Importing standard Qiskit libraries
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
from iqx import *

# Loading your IBM Q account(s)
provider = IBMQ.load_account()

ModuleNotFoundError: No module named 'iqx'

In [2]:
# real device
# Get the least busy backend
'''
from qiskit.providers.ibmq import least_busy
provider = IBMQ.get_provider(hub='ibm-q')
backend = provider.get_backend('ibmq_16_melbourne')
print(backend)
'''

ibmq_16_melbourne


In [3]:
import numpy as np
from qiskit.circuit.library import QFT
from qiskit.aqua import QuantumInstance, aqua_globals
from qiskit.quantum_info import state_fidelity
from qiskit.aqua.algorithms import HHL, NumPyLSsolver
from qiskit.aqua.components.eigs import EigsQPE
from qiskit.aqua.components.reciprocals import LookupRotation
from qiskit.aqua.operators import MatrixOperator
from qiskit.aqua.components.initial_states import Custom

In [4]:
def create_eigs(matrix, num_ancillae, num_time_slices, negative_evals):
    ne_qfts = [None, None]
    if negative_evals:
        num_ancillae += 1
        ne_qfts = [QFT(num_ancillae - 1), QFT(num_ancillae - 1).inverse()]

    return EigsQPE(MatrixOperator(matrix=matrix),
                   QFT(num_ancillae).inverse(),
                   num_time_slices=num_time_slices,
                   num_ancillae=num_ancillae,
                   expansion_mode='suzuki',
                   expansion_order=2,
                   evo_time=None,  # This is t, can set to: np.pi*3/4
                   negative_evals=negative_evals,
                   ne_qfts=ne_qfts)

In [5]:
def preprocess(A, B, row_number, column_number, row_index, column_index):
    #A_new is for the row player, while B_new is for the column player
    A_new = np.zeros((row_number, column_number))
    A_new = np.mat(A_new)
    B_new = np.zeros((row_number, column_number))
    B_new = np.mat(B_new)
    for i in range(row_number):  #initialize A_new and B_new 
        for j in range(column_number):
            A_new[i, j] = A[row_index[i], column_index[j]]
            B_new[i, j] = B[row_index[i], column_index[j]]
    for i in range(row_number - 1):  #construct A_new s.t. A_new * x = b
        A_new[i] = A_new[i] - A_new[i+1]
    A_new[row_number - 1] = [1] * column_number
    B_new = B_new.T
    for i in range(column_number - 1):  #construct B_new s.t. B_new * x = b
        B_new[i] = B_new[i] - B_new[i+1]
    B_new[column_number - 1] = [1] * row_number

    b1 = np.array([0] * column_number)
    b1[column_number - 1] = 1
    b2 = np.array([0] * row_number)
    b2[row_number - 1] = 1
    return (A_new, B_new, b1, b2)

In [6]:
def calculate_matrix(A, B, sequence, m, n):
    # Obtain A, B, b1, b2 where A_new * x = b1, B_new * x = b2
    ########## Preprocess ##########
    sequence_list = [int(i) for i in sequence]
    row  = sequence_list[:m]
    column = sequence_list[m:]
    row_number = sum(row)  #the row number of systems of linear equations
    column_number = sum(column)  ##the column number of systems of linear equations

    row_index = [i for i, v in enumerate(row) if v == 1]
    column_index = [i for i, v in enumerate(column) if v == 1]
    A_new, B_new, b1, b2 = preprocess(A, B, row_number, column_number, row_index, column_index)
    
    flags = [False, False]
    if A_new.shape == (1,1):
        A_new = [[0, A_new[0, 0]], [A_new[0, 0], 0]]
        b1 = [0, b1[0]]
        flags[0] = True
    if B_new.shape == (1,1):
        B_new = [[0, B_new[0, 0]], [B_new[0, 0], 0]]
        b2 = [0, b2[0]]
        flags[1] = True
    matrices = [A_new, B_new]
    vectors = [b1, b2]
    ########## Preprocess End ##########
    
    ########## HHL Algorithm ##########
    sols = []
    for i in range(2):
        orig_size = len(vectors[i])
        matrix, vector, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(matrices[i], vectors[i])
        # Initialize eigenvalue finding module
        eigs = create_eigs(matrix, 3, 50, True)
        num_q, num_a = eigs.get_register_sizes()
        # Initialize initial state module
        init_state = Custom(num_q, state_vector=vector)
        # Initialize reciprocal rotation module
        reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

        algo = HHL(matrix, vector, truncate_powerdim, truncate_hermitian, eigs,
                   init_state, reci, num_q, num_a, orig_size)

        #result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
        #result = algo.run(QuantumInstance(backend))
        result = algo.run(QuantumInstance(provider.get_backend('ibmq_qasm_simulator')))
        if flags[i]:
            sols.append([np.round(result['solution'][0], 5)])
        else:
            sols.append(np.round(result['solution'], 5))
    ########## HHL Algorithm End ##########
    
    print("start checking")
    #check the solution
    #check whether the probability is not negative
    for i in range(row_number):
        if sols[0][i] < 0:
            return False
    for i in range(column_number):
        if sols[1][i] < 0:
            return False
    print("probability >= 0")
    
    #check whether those with zero probabilities have fewer payoffs
    column_strategy = [0] * n
    for i in range(column_number):
        column_strategy[column_index[i]] = sols[1][i]
    row_strategy = [0] * m
    for i in range(row_number):
        row_strategy[row_index[i]] = sols[0][i]
    row_payoff = np.sum(np.array(A[row_index[0]]) * column_strategy)  #positive probability
    for i in range(m):
        temp = 0
        flag = 0
        #consider np.sum(np.array(A[i]) * column_strategy) with positive probability
        for j in range(n):
            if column[j] == 1:
                temp += A[i, j] * column_strategy[flag]
                flag += 1
        if row_payoff - temp  < -10**(-10):
            return False
    C = B.T
    column_payoff = np.sum(np.array(C[column_index[0]]) * row_strategy)  #positive probability
    for i in range(n):
        temp = 0
        flag = 0
        #consider np.sum(np.array(C[i]) * row_strategy) with positive probability
        for j in range(m):
            if row[j] == 1:
                temp += C[i, j] * row_strategy[flag]
                flag += 1
        if column_payoff - temp  < -10**(-10):  
            return False
    print("payoff is correct")
    
    #All is correct: Nash equilibrium
    return (row_strategy, column_strategy)

In [7]:
# Prisoner's Dilemma
A = np.mat('3,0; 5,1')
B = np.mat('3,5; 0,1')
m = 2
n = 2
sequence = '0101'
calculate_matrix(A, B, sequence, m, n)

NameError: name 'provider' is not defined

In [144]:
# An Advertising Game
A = np.mat('6,3; 3,1')
B = np.mat('0,1; 2,1')
m = 2
n = 2
sequence = '1001'
calculate_matrix(A, B, sequence, m, n)

start checking
probability >= 0
payoff is correct


([(0.8682+0j), 0], [0, (0.86949+0j)])

In [146]:
# Test for HHL
#matrix = [[0, -1/2], [-1/2, 0]]
#vector = [2, 2]
matrix = [[0, 1], [1, 0]]
vector = [1, 1]
orig_size = len(vector)
matrix, vector, truncate_powerdim, truncate_hermitian = HHL.matrix_resize(matrix, vector)
# Initialize eigenvalue finding module
eigs = create_eigs(matrix, 3, 50, True)
num_q, num_a = eigs.get_register_sizes()
# Initialize initial state module
init_state = Custom(num_q, state_vector=vector)
# Initialize reciprocal rotation module
reci = LookupRotation(negative_evals=eigs._negative_evals, evo_time=eigs._evo_time)

algo = HHL(matrix, vector, truncate_powerdim, truncate_hermitian, eigs,
           init_state, reci, num_q, num_a, orig_size)

#result = algo.run(QuantumInstance(Aer.get_backend('statevector_simulator')))
#result = algo.run(QuantumInstance(backend))
result = algo.run(QuantumInstance(provider.get_backend('ibmq_qasm_simulator')))
print("Solution:\t\t", np.round(result['solution'], 5))

# Monitoring our job
#from qiskit.tools.monitor import job_monitor
#job_monitor(job)

Solution:		 [0.95121+0.j 1.04652+0.j]
